<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatting with a teachable agent

Conversational assistants based on LLMs can remember the current chat with the user, and can even demonstrate in-context learning of things that the user teaches the assistant during the chat. But these memories and learnings are lost once the chat is over, or when a single chat grows too long for the LLM to handle effectively. In subsequent chats, the user is forced to repeat any necessary instructions over and over.

The optional agent capability called `Teachability` addresses these limitations by persisting user teachings across chat boundaries in long-term memory (a vector database). Memories (called memos) are created and saved to disk throughout a conversation, then loaded from disk later. Instead of copying all the memos into the context window, which would eat up valuable space, individual memos are retrieved into context only as needed. This allows the user to teach many facts, preferences and skills to the teachable agent just once, and have it remember them in later chats.

In making decisions about memo storage and retrieval, `Teachability` calls an instance of `TextAnalyzerAgent` to analyze pieces of text in several different ways. This adds extra LLM calls involving a relatively small number of tokens. These calls can add a few seconds to the time a user waits for a response.

This notebook demonstrates how `Teachability` can be added to an agent so that it can learn facts, preferences, and skills from users. To chat with a teachable agent yourself, run [chat_with_teachable_agent.py](../test/agentchat/contrib/chat_with_teachable_agent.py).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [teachable] option.
```bash
pip install "pyautogen[teachable]"
```

In [1]:
%%capture --no-stderr
%pip install "pyautogen[teachable]"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
# import autogen
# from autogen import ConversableAgent, UserProxyAgent
# from autogen.agentchat.contrib.capabilities.teachability import Teachability

# config_list = autogen.config_list_from_json(
#     env_or_file="OAI_CONFIG_LIST",
#     file_location=".",
#     filter_dict={
#         "model": ["gpt-4", "gpt-4-1106-preview", "gpt4", "gpt-4-32k"],
#     },
# )

# print(config_list[0]["model"])

In [3]:
import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.teachability import Teachability

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["ollama/mistral"],
    },
)

print(config_list[0]["model"])

ollama/mistral


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). After application of the filter shown above, only the gpt-4 models are considered.

The config list may look like the following:
```python
config_list = [
    {
        'model': 'gpt-4-1106-preview',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways if you prefer, e.g., loading from a YAML file.

## Construct Agents
For this walkthrough, we start by creating a teachable agent and resetting its memory store. This deletes any memories from prior conversations that may be stored on disk.

In [4]:
# Start by instantiating any agent that inherits from ConversableAgent.
teachable_agent = ConversableAgent(
    name="teachable_agent",  # The name is flexible, but should not contain spaces to work in group chat.
    llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
)

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
teachability.add_to_agent(teachable_agent)

try:
    from termcolor import colored
except ImportError:

    def colored(x, *args, **kwargs):
        return x


# Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=0,
)


CLEARING MEMORY


/home/twix/miniconda3/envs/genai_samples/lib/python3.10/site-packages/autogen/agentchat/user_proxy_agent.py:82: UserWarning: Using None to signal a default code_execution_config is deprecated. Use {} to use default or False to disable code execution.
  super().__init__(


## Learning new facts
Let's teach the agent some facts it doesn't already know, since they are more recent than GPT-4's training data.

In [5]:
text = "What is the Vicuna model?"
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

What is the Vicuna model?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 I apologize for any confusion, but I don't have enough context or information to provide an accurate answer about the "Vicuna model." Could you please provide some additional details such as the field of study or industry this term may be used in? I will do my best to learn about it if you teach me in our future conversations.

--------------------------------------------------------------------------------


/home/twix/miniconda3/envs/genai_samples/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:838: UserWarning: No summary_method provided or summary_method is not supported: 
  warnings.warn("No summary_method provided or summary_method is not supported: ")


ChatResult(chat_history=[{'content': 'What is the Vicuna model?', 'role': 'assistant'}, {'content': ' I apologize for any confusion, but I don\'t have enough context or information to provide an accurate answer about the "Vicuna model." Could you please provide some additional details such as the field of study or industry this term may be used in? I will do my best to learn about it if you teach me in our future conversations.', 'role': 'user'}], summary='', cost=({'total_cost': 0, 'ollama/mistral': {'cost': 0, 'prompt_tokens': 34, 'completion_tokens': 70, 'total_tokens': 104}}, {'total_cost': 0, 'ollama/mistral': {'cost': 0, 'prompt_tokens': 34, 'completion_tokens': 70, 'total_tokens': 104}}), human_input=[])

In [6]:
text = "Vicuna is a 13B-parameter language model released by Meta."
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachable_agent):

Vicuna is a 13B-parameter language model released by Meta.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 Thank you for sharing the new information about the Vicuna model being a 13B-parameter language model released by Meta. I have committed this new knowledge to my memory. The Vicuna model is a large language model developed and used by Meta (previously Facebook) for various natural language processing tasks. It has 13 billion parameters, making it larger than some other models in the same category, such as Google's BERT model with 110 billion parameters or Microsoft's T5 model with 11 billion parameters. With this new information, I can now answer the question more accurately when someone asks about the Vicuna model in the future. If you have any further questions or need clarification on any topic, feel free to ask!

---------------------------------------------

ChatResult(chat_history=[{'content': 'What is the Vicuna model?', 'role': 'assistant'}, {'content': ' I apologize for any confusion, but I don\'t have enough context or information to provide an accurate answer about the "Vicuna model." Could you please provide some additional details such as the field of study or industry this term may be used in? I will do my best to learn about it if you teach me in our future conversations.', 'role': 'user'}, {'content': 'Vicuna is a 13B-parameter language model released by Meta.', 'role': 'assistant'}, {'content': " Thank you for sharing the new information about the Vicuna model being a 13B-parameter language model released by Meta. I have committed this new knowledge to my memory. The Vicuna model is a large language model developed and used by Meta (previously Facebook) for various natural language processing tasks. It has 13 billion parameters, making it larger than some other models in the same category, such as Google's BERT model with 110 b

In [7]:
text = "What is the Orca model?"
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachable_agent):

What is the Orca model?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 I'm an assistant designed to help answer questions and provide information. Based on the memories I have, the Vicuna model is a 13B-parameter language model released by Meta. However, you asked about the Orca model, and there is no information in my current memories that relates to the Orca model or its connection to the Vicuna model. If you could provide more context or details about the Orca model, I would be happy to try and help with any questions you might have. If the Orca model is also a language model, it might be interesting to compare and contrast its characteristics with those of the Vicuna model if such information becomes available. Let me know if this helps or if there's anything else I can do for you!

--------------------------------------------------------------------------------

ChatResult(chat_history=[{'content': 'What is the Vicuna model?', 'role': 'assistant'}, {'content': ' I apologize for any confusion, but I don\'t have enough context or information to provide an accurate answer about the "Vicuna model." Could you please provide some additional details such as the field of study or industry this term may be used in? I will do my best to learn about it if you teach me in our future conversations.', 'role': 'user'}, {'content': 'Vicuna is a 13B-parameter language model released by Meta.', 'role': 'assistant'}, {'content': " Thank you for sharing the new information about the Vicuna model being a 13B-parameter language model released by Meta. I have committed this new knowledge to my memory. The Vicuna model is a large language model developed and used by Meta (previously Facebook) for various natural language processing tasks. It has 13 billion parameters, making it larger than some other models in the same category, such as Google's BERT model with 110 b

In [8]:
text = "Orca is a 13B-parameter language model developed by Microsoft. It outperforms Vicuna on most tasks."
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachable_agent):

Orca is a 13B-parameter language model developed by Microsoft. It outperforms Vicuna on most tasks.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 Based on the new information you've provided, I have updated my memories with the fact that Orca is also a 13B-parameter language model developed by Microsoft, and it outperforms Vicuna on most tasks. This information will help me answer any future questions about the comparison between Vicuna and Orca models. Thank you for sharing this knowledge! Let me know if there's anything else I can assist you with or if you have any further questions.

--------------------------------------------------------------------------------


ChatResult(chat_history=[{'content': 'What is the Vicuna model?', 'role': 'assistant'}, {'content': ' I apologize for any confusion, but I don\'t have enough context or information to provide an accurate answer about the "Vicuna model." Could you please provide some additional details such as the field of study or industry this term may be used in? I will do my best to learn about it if you teach me in our future conversations.', 'role': 'user'}, {'content': 'Vicuna is a 13B-parameter language model released by Meta.', 'role': 'assistant'}, {'content': " Thank you for sharing the new information about the Vicuna model being a 13B-parameter language model released by Meta. I have committed this new knowledge to my memory. The Vicuna model is a large language model developed and used by Meta (previously Facebook) for various natural language processing tasks. It has 13 billion parameters, making it larger than some other models in the same category, such as Google's BERT model with 110 b

Let's end our first chat here, and start a new chat by clearing the previous chat's history, by passing `clear_history=True` to `initiate_chat`. At this point, a common LLM-based assistant would forget everything from the last chat. But a teachable agent can retrieve memories from its vector DB as needed, allowing it to recall and reason over things that the user taught it in earlier conversations.

In [9]:
text = "How does the Vicuna model compare to the Orca model?"
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

How does the Vicuna model compare to the Orca model?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 Based on the information provided, I can recall that:

1. Both Orca and Vicuna are 13B-parameter language models.
2. Orca was developed by Microsoft and outperforms Vicuna on most tasks.
3. Vicuna is a language model released by Meta, but no specific details or characteristics were given in the text.

Therefore, I can remember that Orca is a more advanced 13B-parameter language model from Microsoft that outperforms Vicuna. However, since there's no explicit information about Vicuna's characteristics or how it differs from Orca aside from their performance, there's not much else to commit to memory in this case.

--------------------------------------------------------------------------------


ChatResult(chat_history=[{'content': 'How does the Vicuna model compare to the Orca model?', 'role': 'assistant'}, {'content': " Based on the information provided, I can recall that:\n\n1. Both Orca and Vicuna are 13B-parameter language models.\n2. Orca was developed by Microsoft and outperforms Vicuna on most tasks.\n3. Vicuna is a language model released by Meta, but no specific details or characteristics were given in the text.\n\nTherefore, I can remember that Orca is a more advanced 13B-parameter language model from Microsoft that outperforms Vicuna. However, since there's no explicit information about Vicuna's characteristics or how it differs from Orca aside from their performance, there's not much else to commit to memory in this case.", 'role': 'user'}], summary='', cost=({'total_cost': 0, 'ollama/mistral': {'cost': 0, 'prompt_tokens': 1501, 'completion_tokens': 627, 'total_tokens': 2128}}, {'total_cost': 0, 'ollama/mistral': {'cost': 0, 'prompt_tokens': 1501, 'completion_toke

## Learning user preferences
Now let's teach the agent some of our preferences. Suppose that we frequently post short summaries of new papers for our team to read, and we want the teachable agent to help us do this faster.

In [10]:
text = """Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework i

ChatResult(chat_history=[{'content': 'Please summarize this abstract.\n\nAutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation\nQingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang\nAutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of t

But that's unstructured. So let's teach the agent our preference for a particular structure.

In [11]:
text = """Please summarize this abstract.
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Please summarize this abstract.
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a gene

ChatResult(chat_history=[{'content': "Please summarize this abstract.\nWhen I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.\n\nAutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation\nQingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang\nAutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen s

That's much better, but will the teachable agent remember these preferences in the future, even for a different paper? Let's start a new chat to find out!

In [12]:
text = """Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general intelligence than previous AI models. We discuss the rising capabilities and implications of these models. We demonstrate that, beyond its mastery of language, GPT-4 can solve novel and difficult tasks that span mathematics, coding, vision, medicine, law, psychology and more, without needing any special prompting. Moreover, in all of these tasks, GPT-4's performance is strikingly close to human-level performance, and often vastly surpasses prior models such as ChatGPT. Given the breadth and depth of GPT-4's capabilities, we believe that it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system. In our exploration of GPT-4, we put special emphasis on discovering its limitations, and we discuss the challenges ahead for advancing towards deeper and more comprehensive versions of AGI, including the possible need for pursuing a new paradigm that moves beyond next-word prediction. We conclude with reflections on societal influences of the recent technological leap and future research directions."""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general intelligence than previous AI models

ChatResult(chat_history=[{'content': "Please summarize this abstract.\n\nSparks of Artificial General Intelligence: Early experiments with GPT-4\nSébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang\nArtificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general intelligence than prev

## Learning new skills
Finally, let's extend the teachable agent's capabilities by teaching it a new skill for accomplishing a challenging type of task. 

The [Sparks of AGI](https://arxiv.org/abs/2303.12712) paper evaluated GPT-4 on math problems like the following, which it could only solve 32% of the time.

In [13]:
text = """Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 To modify exactly one integer on the left hand side of the equation so that the right hand side becomes 99, we need to find which number can be changed to make the left side equal to 101 (since 99 is the result of subtracting 1 from 100), and then adjust the right hand side accordingly.

Let's analyze the given equation:

9 * 4 + 6 * 6 = 72

First, let's see if there is a single number we can modify in such a way that the product of two numbers results in a value close to 13 (the difference between 101 and 72):

- The smallest cha

ChatResult(chat_history=[{'content': 'Consider the identity:\n9 * 4 + 6 * 6 = 72\nCan you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?\n-Let\'s think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".\n', 'role': 'assistant'}, {'content': " To modify exactly one integer on the left hand side of the equation so that the right hand side becomes 99, we need to find which number can be changed to make the left side equal to 101 (since 99 is the result of subtracting 1 from 100), and then adjust the right hand side accordingly.\n\nLet's analyze the given equation:\n\n9 * 4 + 6 * 6 = 72\n\nFirst, let's see if there is a single number we can modify in such a way that the product of two numbers results in a value close to 13 (the difference between 101 and 72):\n\n- The smallest change we could make would be increasing one number by 1, which would result in an 

This time, lets teach the agent a reliable strategy for solving such tasks.

In [14]:
text = """Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachable_agent):

Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".


-

ChatResult(chat_history=[{'content': 'Consider the identity:\n9 * 4 + 6 * 6 = 72\nCan you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?\n-Let\'s think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".\n', 'role': 'assistant'}, {'content': " To modify exactly one integer on the left hand side of the equation so that the right hand side becomes 99, we need to find which number can be changed to make the left side equal to 101 (since 99 is the result of subtracting 1 from 100), and then adjust the right hand side accordingly.\n\nLet's analyze the given equation:\n\n9 * 4 + 6 * 6 = 72\n\nFirst, let's see if there is a single number we can modify in such a way that the product of two numbers results in a value close to 13 (the difference between 101 and 72):\n\n- The smallest change we could make would be increasing one number by 1, which would result in an 

When given this advice, GPT-4 can solve such problems over 95% of the time. But can the teachable agent remember the strategy so the user doesn't have to explain it over and over? As before, let's start a new chat to find out.

In [15]:
text = """Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 Based on the provided memory, here's a plan to modify exactly one integer on the left hand side of the given equation so that the right hand side becomes 99:

1. The original number on the right is 72, and we want it to be 99. So, the difference between the two numbers is 27.
2. We need to find a number on the left hand side that can evenly divide into the difference of 27. Let's examine each number one by one:
   - 9 does not divide evenly into 27.
   - 4 does not divide evenly into 27.
   - 6 divides into 27 twice with a remaind

ChatResult(chat_history=[{'content': 'Consider the identity:\n9 * 4 + 6 * 6 = 72\nCan you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?\n-Let\'s think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".\n', 'role': 'assistant'}, {'content': ' Based on the provided memory, here\'s a plan to modify exactly one integer on the left hand side of the given equation so that the right hand side becomes 99:\n\n1. The original number on the right is 72, and we want it to be 99. So, the difference between the two numbers is 27.\n2. We need to find a number on the left hand side that can evenly divide into the difference of 27. Let\'s examine each number one by one:\n   - 9 does not divide evenly into 27.\n   - 4 does not divide evenly into 27.\n   - 6 divides into 27 twice with a remainder of 0 (i.e., 6 * 4 = 24, and 27 - 24 = 3). Since we need to make the differenc

As a final check, let's test the teachable agent's newly learned skill on a separate instance of the task.

In [16]:
text = """Consider the identity:
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachable_agent):

Consider the identity:
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

 Based on your previous instructions, here's a plan to modify exactly one integer on the left hand side of the given equation so that the right hand side becomes 59:

1. The original number on the right is 87, and we want it to be 59. So, the difference between the two numbers is 28.
2. We need to find a number on the left hand side that can evenly divide into the difference of 28. Let's examine each number one by one:
   - 9 does not divide evenly into 28.
   - 4 divides into 28 four times with a remainder of 0 (i.e., 4 * 7 = 28).

ChatResult(chat_history=[{'content': 'Consider the identity:\n9 * 4 + 6 * 6 = 72\nCan you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?\n-Let\'s think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".\n', 'role': 'assistant'}, {'content': ' Based on the provided memory, here\'s a plan to modify exactly one integer on the left hand side of the given equation so that the right hand side becomes 99:\n\n1. The original number on the right is 72, and we want it to be 99. So, the difference between the two numbers is 27.\n2. We need to find a number on the left hand side that can evenly divide into the difference of 27. Let\'s examine each number one by one:\n   - 9 does not divide evenly into 27.\n   - 4 does not divide evenly into 27.\n   - 6 divides into 27 twice with a remainder of 0 (i.e., 6 * 4 = 24, and 27 - 24 = 3). Since we need to make the differenc